# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Create a full scraping pipeline that involves traversing over many pages of a website, dealing with errors and storing data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [ ]:
# Load the https://www.residentadvisor.net/events page in your browser.

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [ ]:
# Open the inspect element feature in your browser

## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [2]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [3]:
response = requests.get("https://www.residentadvisor.net/events/us/newyork")
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
event_listings = soup.find('div', id="event-listing")

In [5]:
entries = event_listings.findAll('li')
print(len(entries), entries[0])

12 <li><p class="eventDate date"><a href="/events.aspx?ai=8&amp;v=day&amp;mn=6&amp;yr=2020&amp;dy=24"><span>Wed, 24 Jun 2020 /</span></a></p></li>


In [6]:
rows = []
for entry in entries:
    date = entry.find('p', class_="eventDate date")
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df.head()

,0,1,2,3
0,United We Stream x Basement,BASEMENT,"Wed, 24 Jun 2020 /",8.0
1,NYC 90s vs 2K Summer Midnight Yacht Party Jewel,Skyport Marina,"Fri, 26 Jun 2020 /",2.0
2,Just Dance Midnight Yacht Cruise,Harbor Lights Yacht,"Fri, 26 Jun 2020 /",1.0
3,NY Hip Hop vs. Reggae® Summer Midnight Yacht P...,Jewel Yacht,"Fri, 26 Jun 2020 /",NaN
4,[CANCELLED] Wrecked & Carry Nation Pride 2020,Elsewhere,"Sat, 27 Jun 2020 /",119.0


In [7]:
def scrape_events(events_page_url):
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    event_listings = soup.find('div', id="event-listing")
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text
        else:
            continue
    df = pd.DataFrame(rows)
    df.head()
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

In [8]:
scrape_events('https://www.residentadvisor.net/events/us/newyork')

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,United We Stream x Basement,BASEMENT,"Wed, 24 Jun 2020 /",8.0
1,NYC 90s vs 2K Summer Midnight Yacht Party Jewel,Skyport Marina,"Fri, 26 Jun 2020 /",2.0
2,Just Dance Midnight Yacht Cruise,Harbor Lights Yacht,"Fri, 26 Jun 2020 /",1.0
3,NY Hip Hop vs. Reggae® Summer Midnight Yacht P...,Jewel Yacht,"Fri, 26 Jun 2020 /",NaN
4,[CANCELLED] Wrecked & Carry Nation Pride 2020,Elsewhere,"Sat, 27 Jun 2020 /",119.0
5,[CANCELLED] 718 Sessions Boat Party 2020,Circle Line Cruises,"Sun, 28 Jun 2020 /",27.0
6,[CANCELLED] 718 Sessions Boat Party 2020,Circle Line Cruises,"Mon, 29 Jun 2020 /",27.0


In [9]:
scrape_events('https://www.residentadvisor.net/events/us/dallasfortworth')

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Tinlicker,It'll Do,"Fri, 26 Jun 2020 /",2


In [20]:
scrape_events('https://www.residentadvisor.net/events/us/austin')



,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Klubhaus 006,TBA - Austin,"Sat, 27 Jun 2020 /",19


## Write a Function to Retrieve the URL for the Next Page

In [11]:
soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']

'/events/us/newyork/week/2020-06-30'

In [12]:
def next_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    url_ext = soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    return next_page_url

In [13]:
next_page('https://www.residentadvisor.net/events/us/newyork/week/2020-06-30')

'https://www.residentadvisor.net/events/us/newyork/week/2020-07-07'

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [30]:
#Your code here
dfs = [0,4]
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/newyork"
while total_rows <= 1000:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
    time.sleep(.2)
df = pd.concat(dfs)
df = df.iloc[:1000]
print(len(df))
df.head()

KeyboardInterrupt: 

## Summary 

Congratulations! In this lab, you successfully developed a pipeline to scrape a website for concert event information!